In [3]:
import pulp
from haversine import haversine

# 3 x 3 represented as linear
ncities=68

locations=[(90,0),
(72.882963737,-106.806316244),
(71.9160703549,-106.790825512),
(69.6143699264,-106.776707895),
(67.29182297,-106.994305651),
(67.0493563631,-106.785698654),
(66.5045693397,-106.859494197),
(66.3983702853,-106.839366548),
(66.0359981983,-106.849053515),
(64.7525191421,-106.762028917),
(63.6991506071,-106.749346366),
(63.3115801175,-106.805200516),
(62.0777190775,-106.73928736),
(61.2084690259,-106.84776886),
(60.622466454,-106.888576669),
(59.1275190853,-106.732937752),
(58.1309909187,-106.867865498),
(57.420082929,-106.841132015),
(56.0108357219,-106.999845901),
(55.7715093342,-106.730834887),
(55.6900084975,-106.869155531),
(54.9734923753,-106.997071573),
(53.4378536891,-106.930512807),
(52.5188181322,-106.911045673),
(50.9405095276,-106.808712877),
(47.8976275036,-106.83549647),
(47.7393825937,-106.849850083),
(47.335554105,-106.734743591),
(46.6109464861,-106.756280413),
(45.2370788867,-106.977431926),
(45.0412378868,-106.902656844),
(43.0315063247,-106.860951135),
(42.5414610858,-106.878837374),
(41.4911969975,-106.959636236),
(40.9997179856,-106.88037065),
(40.0575584032,-106.939570733),
(39.9797998931,-106.907024825),
(39.637622186,-106.896289312),
(39.1185851047,-106.830315369),
(38.9342631243,-106.839671685),
(38.2283410262,-106.993755344),
(38.0441704393,-106.738090369),
(37.2545664601,-106.760129341),
(36.8368843991,-106.929489949),
(35.0819214111,-106.824429456),
(34.7871322657,-106.807555179),
(33.7070564728,-106.938183555),
(32.1986875921,-106.732014961),
(31.9442020896,-106.801924154),
(31.6537777338,-106.931606555),
(30.030780887,-106.924553075),
(29.8204269966,-106.877093359),
(29.2444424264,-106.777731348),
(29.1994654037,-106.801800424),
(28.4249860024,-106.950594707),
(27.9474130474,-106.919340434),
(27.8820318412,-106.898221237),
(27.8536451475,-106.978266059),
(27.3620275641,-106.795977988),
(27.1055822661,-106.759713208),
(26.1530027273,-106.876876728),
(25.8026387548,-106.866190969),
(25.7451222775,-106.730702988),
(25.0997805051,-106.965322948),
(25.0195060828,-106.799138013),
(24.7713001538,-106.908232271),
(24.4461683106,-106.863524053),
(24.1018287359,-106.962846223)
]

C=[haversine(locations[i],locations[j]) for i in range(ncities) for j in range(ncities)]

bestsol=sum([C[ncities*i+i+1] for i in range(ncities-1)])+C[ncities*(ncities-1)]
print bestsol

14727.863687


In [4]:
#nfix=range(1,31) #Optimize first 10 cities

cities=range(ncities*ncities)
sequence=range(ncities)

xx=[0 for i in range(ncities) for j in range(ncities)]
for i in range(ncities-1):
    xx[ncities*i+i+1]=1

xx[ncities*(ncities-1)]=1
    
X = pulp.LpVariable.dicts('trips', cities,lowBound=0, upBound=1, cat='Integer')
#X.setInitialValue(xx)

U = pulp.LpVariable.dicts('seq', sequence,lowBound=0,upBound=ncities-1,cat='Integer')
#U.setInitialValue(range(len(ncities)))

#S = pulp.LpVariables.dicts('switch',sequence,lowBound=0,upBound=1,cat='Integer')

tsp_model = pulp.LpProblem("TSP Model", pulp.LpMinimize)

tsp_model += sum([C[i]*X[i] for i in range(len(C))])

for i in range(ncities):
    tsp_model += sum([X[ncities*i+j] for j in range(ncities)])==1

for j in range(ncities):
    tsp_model += sum([X[ncities*i+j] for i in range(ncities)])==1

for i in range(ncities):
    tsp_model += X[ncities*i+i]==0
    
#Outperform current solution
#tsp_model += sum([C[i]*X[i] for i in range(len(C))]) <= bestsol

#Max 5 changes
#tsp_model += sum(X[ncities*i+j]-xx[ncities*i+j] for i in range(ncities) for j in range(ncities))>=-2
#tsp_model += sum(X[ncities*i+j]-xx[ncities*i+j] for i in range(ncities) for j in range(ncities))<= 2

##Max 5 shifts from current sequence
#for i in range(1,ncities):
#    tsp_model += U[i] >=max(i-5,1)

#for i in range(1,ncities):
#    tsp_model += U[i] <=min(ncities-1,i+5)


#Subtour elimination and Sequence
tsp_model += U[0] == 0

for i in range(1,ncities):
    tsp_model += U[i]>=1
    
for i in range(1,ncities):
    for j in range(1,ncities):
        if i<>j:
            tsp_model += U[i]-U[j]+(ncities-1)*X[ncities*i+j] <= (ncities-2)


tsp_model.solve(pulp.GLPK(options=['--dual','--bfs','--cuts','--tmlim', '180']))


1

In [ ]:
import pulp
##Traveling Salesman Problem (Subtour Eliminations)
def tsp_solve_mtz(ll):
    
    locations=[north_pole]+[ll[i][1] for i in range(len(ll))]
    weights=[0]+[ll[i][2] for i in range(len(ll))]
    tot_weight=sum(weights)
    
    ncities=len(locations)
    cities = range(ncities*ncities)
    C=[haversine(locations[i],locations[j])*weights[j] for i in range(ncities) for j in range(ncities)]

    X = pulp.LpVariable.dicts('trips', cities,lowBound=0, upBound=1, cat='Integer')
    
    U = pulp.LpVariable.dicts('seq', sequence,lowBound=0,upBound=ncities-1,cat='Integer')

    tsp_model = pulp.LpProblem("TSP Model", pulp.LpMinimize)

    #Without weights
    #tsp_model += sum([C[i]*X[i] for i in range(len(C))])
    
    #Wrong formulation 1
    #tsp_model += tot_weight*sum([C[i]*X[i] for i in range(len(C))]) - sum(weights[k]*C[ncities*k+j]*X[ncities*k+j] for k in range(1,ncities) for j in range(ncities))

    ##Approximate formulation
    tsp_model += sum([C[ncities*i+j]*(10+tot_weight-weights[i])*X[ncities*i+j] for i in range(ncities) for j in range(ncities)])
    
    for i in range(ncities):
        tsp_model += sum([X[ncities*i+j] for j in range(ncities)])==1

    for j in range(ncities):
        tsp_model += sum([X[ncities*i+j] for i in range(ncities)])==1

    for i in range(ncities):
        tsp_model += X[ncities*i+i]==0
        
    #Subtour elimination and Sequence
    tsp_model += U[0] == 0

    for i in range(1,ncities):
        tsp_model += U[i]>=1

    for i in range(1,ncities):
        for j in range(1,ncities):
            if i<>j:
                tsp_model += U[i]-U[j]+(ncities-1)*X[ncities*i+j] <= (ncities-2)


    tsp_model.writeLP("tsp_model.lp")

    tsp_model.solve(pulp.GLPK(options=['--dual','--bfs','--cuts','--tmlim', '180']))

    print pulp.LpStatus[tsp_model.status]

    ##Re-arrange ll
    lopt=list()
    snode=0
    while True:
        for j in range(ncities):
            if (X[ncities*snode+j].value()>0):
                #print 'Next Node',j
                if j>0:
                    lopt.append(ll[j-1])
                break
        
        snode=j
        if snode==0:
            break
        
    #print lopt
    return lopt